### MCMF(최소 비용 최대 유량)
- 최소 비용을 이용하여 최대 유량을 흘리는 문제.
- 즉 기존의 유량 그래프 내의 간선에 비용 정보가 추가된 문제이다.
- 유량이 최대일 때 그 비용의 합을 최소로 하는 문제이다.

### SPFA(Shortest Path Faster Algorithm)
- Ford-Fulkerson, Edmond-Karp 알고리즘과 비슷하게 매번 경로를 찾아 더 이상의 경로가 없을 때 까지 유량을 흘리는데, 그 경로를 최단 경로로 찾는다.
  - 여기서 최단경로란, `Source에서 Sink로 가는 최소 비용의 거리` = 간선의 가중치가 비용이라고 생각하고 그 최소비용을 찾는다.
- 매번 최단 경로를 찾고, 그 경로를 통해 흐를 수 있는 최대 유량을 찾은 후 흘려보내면서, 그 결과에 이번에 찾은 경로의 `비용 합` * `유량`을 더해준다.
- 보통 최단 경로를 변형한 알고리즘을 사용하지만, 적어도 음의 가중치를 사용할 수 있어야 한다.
  - 따라서 Bellman-Ford를 변형한 SPFA를 사용한다.
  - 시간복잡도는 $\Omicron(VEf^*)$ 이다.
- 최소 비용으로 이분매칭을 하는 경우는 헝가리안 알고리즘을 사용한다고 한다.

### 구현
- 좀 많이 느리긴 한데, $1 \le V \le 800$ 의 범위에서 TLE 없이 문제를 풀 수 있었다.
- 주석은 11408(열혈강호 5) 참고

In [ ]:
import collections
INF = 1 << 30
class MCMF:
  def __init__(self, n) :
    self.len = n
    self.G = [[] for _ in range(n)]
    self.C = [[0] * n for _ in range(n)]
    self.F = [[0] * n for _ in range(n)]
    self.W = [[0] * n for _ in range(n)]

  def addEdge(self, u, v, c, w) :
    self.G[u].append(v)
    self.G[v].append(u)
    self.C[u][v] = c
    self.W[u][v] = w
    self.W[v][u] = -w

  def SPFA(self, src, snk):
    mc = mf = 0
    while True :
      vis = [False] * self.len
      dist = [INF] * self.len
      par = [-1] * self.len

      Q = collections.deque([src])
      dist[src] = 0
      vis[src] = True
      while Q :
        u = Q.popleft()
        vis[u] = False
        for v in self.G[u] :
          if self.C[u][v] - self.F[u][v] > 0 and dist[v] > dist[u] + self.W[u][v] :
            dist[v] = dist[u] + self.W[u][v]
            par[v] = u
            if not vis[v] :
              vis[v] = True
              Q.append(v)

      if par[snk] == -1 : break

      flow = INF
      u = snk
      while u != src :
        flow = min(flow, self.C[par[u]][u] - self.F[par[u]][u])
        u = par[u]
      
      u = snk
      while u != src :
        mc += flow * self.W[par[u]][u]
        self.F[par[u]][u] += flow
        self.F[u][par[u]] -= flow
        u = par[u]

      mf += 1

    return mc, mf

### TODO
- 인접 List SFPA?
- 인기가 없는 알고리즘인지 템플릿 구하기가 영 쉽지 않다
- https://www.teferi.net/ps/%EC%B5%9C%EC%86%8C_%EB%B9%84%EC%9A%A9_%EC%B5%9C%EB%8C%80_%EC%9C%A0%EB%9F%89
- https://cp-algorithms.com/graph/min_cost_flow.html#implementation